In [1]:
import os 

In [2]:
os.chdir("F:\\Analytics Vidhya Al & ML\\NLP\\Spam Classification")

In [3]:
import pandas as pd

In [4]:
data=pd.read_csv("spam.csv",encoding="latin-1")

In [5]:
data.head()

,ï»¿label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
import string
punctuation=string.punctuation

from nltk.corpus import stopwords
stopword_list=stopwords.words("english")

from nltk.stem.wordnet import WordNetLemmatizer
lem=WordNetLemmatizer()
def _clean(text):
    
    cleaned_text=text.lower()
    cleaned_text="".join(c for c in cleaned_text if c not in punctuation)
    
    words= cleaned_text.split()
    words=[w for w in words if w not in stopword_list]
    
    words=[lem.lemmatize(words,"v") for words in words]
    words=[lem.lemmatize(words,"n") for words in words]
    
    cleaned_text=" ".join(words)
    
    
    return cleaned_text
_clean("I will be playing Cricket Today !!!@#")

'play cricket today'

In [7]:
data["cleaned"]=data['text'].apply(_clean)

In [8]:
data.head()

,ï»¿label,text,cleaned
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think go usf live around though


In [9]:
##Feature Engineering 

#meta feature : some of the attributes or counts that are associated with the text data

data['word_count']=data['text'].apply(lambda x : len(x.split()))
data['word_count_cleaned']=data['cleaned'].apply(lambda x : len(x.split()))
data['char_count']=data['text'].apply(lambda x : len(x))
data['char_count_without_spaces']=data['text'].apply(lambda x: len(x.replace(" ","")))
data['num_digit']=data['text'].apply(lambda x : sum([1 if w.isdigit else 0 for w in x.split()]))

In [10]:
data.head()

,ï»¿label,text,cleaned,word_count,word_count_cleaned,char_count,char_count_without_spaces,num_digit
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...,20,16,111,92,20
1,ham,Ok lar... Joking wif u oni...,ok lar joke wif u oni,6,6,29,24,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,28,23,155,128,28
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say,11,9,49,39,11
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think go usf live around though,13,8,61,49,13


In [12]:
#other type of features can be NLP features 

pos_dic={"noun":['NNP','NN','NNS','NNPS'],'verb':['VBZ','VB','VBD','VBN']}
#defining the function for the count of words of the POS
import nltk
def pos_check(text,family):
    tags=nltk.pos_tag(nltk.word_tokenize(text))
    count=0
    for tag in tags: #it will return list of tags and corresponding POStag
        tag=tag[1] # to obtain the proper pos tag we define tag[1]
        if tag in pos_dic[family]: # if the pos is present in the particular family then increase the count by 1
            count+=1
    return count

#pos_check("They are playing the ground","noun")
data['noun_count']=data['text'].apply(lambda x: pos_check(x,"noun"))
data['verb_count']=data['text'].apply(lambda x: pos_check(x,"verb"))

In [13]:
data.head()

,ï»¿label,text,cleaned,word_count,word_count_cleaned,char_count,char_count_without_spaces,num_digit,noun_count,verb_count
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...,20,16,111,92,20,10,1
1,ham,Ok lar... Joking wif u oni...,ok lar joke wif u oni,6,6,29,24,6,4,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,28,23,155,128,28,13,3
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say,11,9,49,39,11,3,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think go usf live around though,13,8,61,49,13,1,4


In [14]:
# one of the ideas to use vectors are count as the feature like count of every word in the document or in a corpus

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer


In [14]:
#count_vectors


In [15]:
word_tfidf=TfidfVectorizer(max_features=500)
word_tfidf.fit(data['cleaned'].values)
word_vectors=word_tfidf.transform(data['cleaned'].values)

In [16]:
tfidf=dict(zip(word_tfidf.get_feature_names(),word_tfidf.idf_))
tfidf_df=pd.DataFrame(columns=['word_tfidf']).from_dict(tfidf,orient='index')
tfidf_df.columns=['word_tfidf']
tfidf_df

,word_tfidf
10,6.581166
100,5.936809
1000,5.754488
150,5.888019
150p,6.406813
...,...
yet,5.713666
yo,6.099328
youre,5.655397
yr,6.581166


In [17]:
from scipy.sparse import hstack,csr_matrix

In [18]:
data.columns

Index(['ï»¿label', 'text', 'cleaned', 'word_count', 'word_count_cleaned',
       'char_count', 'char_count_without_spaces', 'num_digit', 'noun_count',
       'verb_count'],
      dtype='object')

In [19]:
meta_features=['word_count', 'word_count_cleaned',
       'char_count', 'char_count_without_spaces', 'num_digit', 'noun_count',
       'verb_count']

In [20]:
feature_set=data[meta_features]

In [21]:
train=hstack([word_vectors,csr_matrix(feature_set)],"csr")

In [22]:
train

<5572x507 sparse matrix of type '<class 'numpy.float64'>'
	with 65862 stored elements in Compressed Sparse Row format>

In [23]:
from sklearn.preprocessing import LabelEncoder

target=data['ï»¿label'].values
target=LabelEncoder().fit_transform(target)

In [24]:
target

array([0, 0, 1, ..., 0, 0, 0])

In [25]:
from sklearn.model_selection import train_test_split

train_x,valid_x,train_y,valid_y=train_test_split(train,target)


In [26]:
train_x.shape,valid_x.shape

((4179, 507), (1393, 507))

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import naive_bayes
from sklearn import ensemble
from sklearn.metrics import accuracy_score

In [28]:
model=naive_bayes.MultinomialNB()
model.fit(train_x,train_y)
preds=model.predict(valid_x)
accuracy_score(preds,valid_y)

0.9655419956927495

In [29]:
model=LogisticRegression()
model.fit(train_x,train_y)
preds=model.predict(valid_x)
accuracy_score(preds,valid_y)

C:\Users\Saikrizna\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9511844938980617

In [30]:
model=svm.SVC()
model.fit(train_x,train_y)
preds=model.predict(valid_x)
accuracy_score(preds,valid_y)

0.9210337401292176

In [31]:
model=ensemble.ExtraTreesClassifier()
model.fit(train_x,train_y)
preds=model.predict(valid_x)
accuracy_score(preds,valid_y)

0.9813352476669059

In [32]:
import numpy as np

embeddings_index={}
for i , line in enumerate(open('pretrained.vec',encoding="utf8")): #opening file & iterating one by one 
    if i ==0:
        continue
    value = line.split()#splitting the each line 
    print(line)
    break
    embeddings_index[value[0]]=np.array(value[1:],dtype='float32') # all the lines that contains first element as word & corresponding word vector
                                                   # and converting them to numpy array instead of raw strings

, -0.0282 -0.0557 -0.0451 -0.0434 0.0712 -0.0855 -0.1085 -0.0561 -0.4523 -0.0202 0.0975 0.1047 0.1962 -0.0693 0.0213 -0.0235 0.1336 -0.0420 -0.0564 -0.0798 0.0424 -0.0409 -0.0536 -0.0252 0.0135 0.0064 0.1235 0.0461 0.0120 -0.0372 0.0650 0.0041 -0.1074 -0.0263 0.1133 -0.0029 0.0671 0.1065 0.0234 -0.0160 0.0070 0.4355 -0.0752 -0.4328 0.0457 0.0604 -0.0740 -0.0055 -0.0089 -0.2926 -0.0545 -0.1519 0.0990 -0.0193 -0.0050 0.0511 0.0404 0.1023 -0.0128 0.0488 -0.1567 -0.0759 -0.0190 0.1442 0.0047 -0.0186 0.0140 -0.0385 -0.0853 0.1572 0.1770 0.0084 -0.0250 -0.1145 -0.0663 -0.1244 -0.3977 -0.0124 -0.4586 -0.0220 0.5746 0.0218 -0.0754 0.0099 0.0397 -0.0154 0.0424 -0.0150 -0.0016 0.0305 0.0101 0.2266 0.1394 0.0189 0.0069 0.0394 0.0355 -0.0111 -0.0687 -0.0078 0.0224 0.0817 -0.1949 0.0001 0.4047 -0.0237 -0.0656 -0.0684 0.0233 0.0438 0.1203 -0.0276 0.0416 0.0114 -0.4529 0.1538 0.1323 -0.0186 -0.0914 -0.0312 0.1051 0.0212 0.0798 -0.0104 -0.0206 -0.0025 0.0043 -0.0378 0.2689 0.0747 -0.0418 -0.0048 -0.03

In [33]:
# converting text data into word embedding 
from keras.preprocessing import text, sequence 

token = text.Tokenizer()
token.fit_on_texts(data['text'])
word_index=token.word_index

#Next step is to convert the text into sequence of tokens & also pad them to make sure equal length of vectors are obtained
train_x,valid_x,train_y,valid_y=train_test_split(data['text'],target)


train_x=sequence.pad_sequences(token.texts_to_sequences(train_x),maxlen=70)
valid_x=sequence.pad_sequences(token.texts_to_sequences(valid_x),maxlen=70)

embedding_matrix= np.zeros((len(word_index)+1, 300))
for word, i in word_index.items():
    embedding_vector=embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i]=embedding_vector

Using TensorFlow backend.
C:\Users\Saikrizna\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Saikrizna\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Saikrizna\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Saikrizna\A

In [36]:
embedding_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [47]:
def train_model(classifier,feature_vector_train,label,feature_vector_val,valid_y):
    classifier.fit(feature_vector_train,label)
    predictions=classifier.predict(feature_vector_val)
    predictions=predictions.argmax(axis=-1)
    return accuracy_score(predictions,valid_y)

In [55]:
from keras import layers, models,optimizers

def create_cnn():
    
    input_layer =layers.Input((70,))
    
    embedding_layer=layers.Embedding(len(word_index)+1,300,weights=[embedding_matrix],trainable=False)(input_layer)
    
    conv_layer = layers.Convolution1D(100,3,activation='relu')(embedding_layer)
    
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)
    
    output_layer = layers.Dense(50,activation='relu')(pooling_layer)
    output_layer = layers.Dropout(0.25)(output_layer)
    output_layer = layers.Dense(1,activation='sigmoid')(output_layer)
    
    model=models.Model(inputs=input_layer,output=output_layer)
    model.compile(optimizer=optimizers.Adam(),loss='binary_crossentropy')
    return model

In [56]:
classifier =create_cnn()
train_model(classifier,train_x,train_y,valid_x,valid_y)

C:\Users\Saikrizna\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Epoch 1/1
4179/4179 [==============================] - 5s 1ms/step - loss: 0.6711


0.8815506101938263